<a href="https://colab.research.google.com/github/maiaufrrj/superstore_data/blob/main/modelo_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

referencia: 
https://www.kaggle.com/andreshg/timeseries-analysis-a-complete-guide
https://support.sisense.com/kb/en/article/visualizing-forecasting-data-prophet-in-plotly-%E2%80%94-python

In [ ]:
!pip install fbprophet
!pip install pystan
!pip install --upgrade plotly

In [ ]:
# importar os pacotes necessários
import pandas as pd
import plotly.express as px
import numpy as np

#importar o arquivo csv do meu repositório público no github 
dataset='https://raw.githubusercontent.com/maiaufrrj/superstore_data/main/superstore_dataset2011-2015.csv'
df = pd.read_csv(dataset, encoding= 'unicode_escape')

# visualizar início dos dados
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,Country,Postal Code,Market,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,42433,AG-2011-2040,1/1/2011,6/1/2011,Standard Class,TB-11280,Toby Braunhardt,Consumer,Constantine,Constantine,Algeria,NaN,Africa,Africa,OFF-TEN-10000025,Office Supplies,Storage,"Tenex Lockers, Blue",408.300,2,0.0,106.140,35.46,Medium
1,22253,IN-2011-47883,1/1/2011,8/1/2011,Standard Class,JH-15985,Joseph Holt,Consumer,Wagga Wagga,New South Wales,Australia,NaN,APAC,Oceania,OFF-SU-10000618,Office Supplies,Supplies,"Acme Trimmer, High Speed",120.366,3,0.1,36.036,9.72,Medium
2,48883,HU-2011-1220,1/1/2011,5/1/2011,Second Class,AT-735,Annie Thurman,Consumer,Budapest,Budapest,Hungary,NaN,EMEA,EMEA,OFF-TEN-10001585,Office Supplies,Storage,"Tenex Box, Single Width",66.120,4,0.0,29.640,8.17,High
3,11731,IT-2011-3647632,1/1/2011,5/1/2011,Second Class,EM-14140,Eugene Moren,Home Office,Stockholm,Stockholm,Sweden,NaN,EU,North,OFF-PA-10001492,Office Supplies,Paper,"Enermax Note Cards, Premium",44.865,3,0.5,-26.055,4.82,High
4,22255,IN-2011-47883,1/1/2011,8/1/2011,Standard Class,JH-15985,Joseph Holt,Consumer,Wagga Wagga,New South Wales,Australia,NaN,APAC,Oceania,FUR-FU-10003447,Furniture,Furnishings,"Eldon Light Bulb, Duo Pack",113.670,5,0.1,37.770,4.70,Medium


In [ ]:
#criando dicionário para entender cada recurso disponível e posteriormente modificar dataframe

columns = {'Row ID':'id_linha',
           'Order ID': 'id_pedido', 
           'Order Date': 'data_pedido',
           'Ship Date': 'data_envio',
           'Ship Mode': 'modalidade_frete',
           'Customer ID': 'id_cliente',
           'Customer Name': 'cliente',
           'Segment': 'segmento',
           'City': 'cidade',
           'State': 'estado',
           'Country': 'país',
           'Postal Code': 'cod_postal',
           'Market': 'mercado',
           'Region': 'regiao',
           'Product ID': 'id_produto',
           'Category': 'categoria_produto',
           'Sub-Category': 'subcategoria_produto',
           'Product Name': 'produto',
           'Sales': 'vendas',
           'Quantity': 'quantidade',
           'Discount': 'desconto',
           'Profit': 'lucro',
           'Shipping Cost': 'custo_envio',
           'Order Priority': 'prioridade'
           }
#trocando nomes das colunas e selecionando coluna de índice
df=df.rename(columns=columns).set_index('id_pedido')      

In [ ]:
df.head()

,id_linha,data_pedido,data_envio,modalidade_frete,id_cliente,cliente,segmento,cidade,estado,país,cod_postal,mercado,regiao,id_produto,categoria_produto,subcategoria_produto,produto,vendas,quantidade,desconto,lucro,custo_envio,prioridade
id_pedido,,,,,,,,,,,,,,,,,,,,,,,
AG-2011-2040,42433,1/1/2011,6/1/2011,Standard Class,TB-11280,Toby Braunhardt,Consumer,Constantine,Constantine,Algeria,NaN,Africa,Africa,OFF-TEN-10000025,Office Supplies,Storage,"Tenex Lockers, Blue",408.300,2,0.0,106.140,35.46,Medium
IN-2011-47883,22253,1/1/2011,8/1/2011,Standard Class,JH-15985,Joseph Holt,Consumer,Wagga Wagga,New South Wales,Australia,NaN,APAC,Oceania,OFF-SU-10000618,Office Supplies,Supplies,"Acme Trimmer, High Speed",120.366,3,0.1,36.036,9.72,Medium
HU-2011-1220,48883,1/1/2011,5/1/2011,Second Class,AT-735,Annie Thurman,Consumer,Budapest,Budapest,Hungary,NaN,EMEA,EMEA,OFF-TEN-10001585,Office Supplies,Storage,"Tenex Box, Single Width",66.120,4,0.0,29.640,8.17,High
IT-2011-3647632,11731,1/1/2011,5/1/2011,Second Class,EM-14140,Eugene Moren,Home Office,Stockholm,Stockholm,Sweden,NaN,EU,North,OFF-PA-10001492,Office Supplies,Paper,"Enermax Note Cards, Premium",44.865,3,0.5,-26.055,4.82,High
IN-2011-47883,22255,1/1/2011,8/1/2011,Standard Class,JH-15985,Joseph Holt,Consumer,Wagga Wagga,New South Wales,Australia,NaN,APAC,Oceania,FUR-FU-10003447,Furniture,Furnishings,"Eldon Light Bulb, Duo Pack",113.670,5,0.1,37.770,4.70,Medium


In [ ]:
df['vendas_unit']=df['vendas']/df['quantidade']
df['custo_envio_unit']=df['custo_envio']/df['quantidade']
df['lucro_unit']=df['lucro']/df['quantidade']
#df.sample(50)

,id_linha,data_pedido,data_envio,modalidade_frete,id_cliente,cliente,segmento,cidade,estado,país,cod_postal,mercado,regiao,id_produto,categoria_produto,subcategoria_produto,produto,vendas,quantidade,desconto,lucro,custo_envio,prioridade,vendas_unit,custo_envio_unit,lucro_unit
id_pedido,,,,,,,,,,,,,,,,,,,,,,,,,,
ES-2014-2538638,16761,15-10-2014,17-10-2014,Second Class,CS-11860,Cari Schnelling,Consumer,Vienna,Vienna,Austria,NaN,EU,Central,OFF-ST-10000624,Office Supplies,Storage,"Eldon File Cart, Single Width",1026.960,8,0.00,441.3600,326.35,Critical,128.3700,40.793750,55.1700
US-2012-128797,9130,11/6/2012,15-06-2012,Standard Class,KA-16525,Kelly Andreada,Consumer,Saltillo,Coahuila,Mexico,NaN,LATAM,North,OFF-ST-10001172,Office Supplies,Storage,"Fellowes Trays, Wire Frame",112.920,3,0.00,12.4200,2.59,Medium,37.6400,0.863333,4.1400
US-2012-103814,9367,5/11/2012,11/11/2012,Standard Class,HD-14785,Harold Dahlen,Home Office,San Miguelito,Panama,Panama,NaN,LATAM,Central,OFF-LA-10002972,Office Supplies,Labels,"Avery File Folder Labels, Alphabetical",10.728,3,0.40,-0.9120,0.85,Medium,3.5760,0.283333,-0.3040
CA-2013-144785,40975,19-04-2013,24-04-2013,Standard Class,CS-12490,Cindy Schnelling,Corporate,New York City,New York,United States,10009.0,US,East,OFF-SU-10004290,Office Supplies,Supplies,"Acme Design Line 8"" Stainless Steel Bent Sciss...",6.840,1,0.00,1.8468,0.55,Medium,6.8400,0.550000,1.8468
CA-2014-105991,36584,6/5/2014,7/5/2014,First Class,LH-17020,Lisa Hazard,Consumer,Louisville,Colorado,United States,80027.0,US,West,FUR-BO-10004467,Furniture,Bookcases,Bestar Classic Bookcase,89.991,3,0.70,-152.9847,8.44,High,29.9970,2.813333,-50.9949
IT-2014-2277753,14873,14-01-2014,16-01-2014,Second Class,AG-10765,Anthony Garverick,Home Office,Florence,Tuscany,Italy,NaN,EU,South,FUR-CH-10001039,Furniture,Chairs,"Harbour Creations Rocking Chair, Adjustable",358.992,6,0.60,-251.3880,30.31,Medium,59.8320,5.051667,-41.8980
MX-2014-164889,2681,8/9/2014,12/9/2014,Second Class,JE-15745,Joel Eaton,Consumer,Camagüey,Camagüey,Cuba,NaN,LATAM,Caribbean,OFF-LA-10000633,Office Supplies,Labels,"Novimex File Folder Labels, Laser Printer Comp...",11.000,2,0.00,0.3200,2.33,High,5.5000,1.165000,0.1600
IT-2012-1191900,14613,19-06-2012,25-06-2012,Standard Class,JD-16060,Julia Dunbar,Consumer,Toulouse,Midi-Pyrénées,France,NaN,EU,Central,TEC-PH-10001574,Technology,Phones,"Nokia Audio Dock, Cordless",118.713,2,0.65,-152.6670,10.76,Medium,59.3565,5.380000,-76.3335
ES-2014-1085728,20137,18-12-2014,20-12-2014,First Class,CL-11890,Carl Ludwig,Consumer,Vigevano,Lombardy,Italy,NaN,EU,South,OFF-LA-10001793,Office Supplies,Labels,"Avery Round Labels, Alphabetical",27.840,4,0.00,6.1200,4.41,High,6.9600,1.102500,1.5300


In [ ]:
#criar dataset resumido em vendas mensais
df2=df
df2.index = pd.to_datetime(df2['data_pedido'])

df2['mes_ano'] = pd.to_datetime(df2['data_pedido']).apply(lambda x: '{month}-{year}'.format(month=x.month, year=x.year))

df_vendas = df2.groupby('mes_ano')['vendas'].sum()
df_vendas.to_frame()

df_vendas = pd.DataFrame({'mes_ano':df_vendas.index, 'vendas':df_vendas.values})
df_vendas['ano'] = pd.DatetimeIndex(df_vendas['mes_ano']).year
df_vendas['mes'] = pd.DatetimeIndex(df_vendas['mes_ano']).month

#último dia do mês
from pandas.tseries.offsets import MonthEnd
df_vendas['ultimo_dia_mes'] = pd.to_datetime(df_vendas['mes_ano']) + MonthEnd(1)

df_vendas = df_vendas.sort_values(['ano', 'mes'], ascending = (True, True))
df_vendas.set_index('mes_ano');
#df_vendas.head(50)

,vendas,ano,mes,ultimo_dia_mes
mes_ano,,,,
1-2011,138241.30042,2011,1,2011-01-31
2-2011,134969.94086,2011,2,2011-02-28
3-2011,171455.59372,2011,3,2011-03-31
4-2011,128833.47034,2011,4,2011-04-30
5-2011,148146.72092,2011,5,2011-05-31
6-2011,189338.43966,2011,6,2011-06-30
7-2011,162034.69756,2011,7,2011-07-31
8-2011,219223.49524,2011,8,2011-08-31
9-2011,255237.89698,2011,9,2011-09-30


In [ ]:
#gráfico para visualizar evolução de vendas
#fig = px.line(df_vendas, x='ultimo_dia_mes', y="vendas")
#fig.show()

In [ ]:
train_size = int(0.85 * len(df_vendas))
test_size = len(df_vendas) - train_size

univariate_df = df_vendas[['ultimo_dia_mes', 'vendas']].copy()
univariate_df.columns = ['ds', 'y']

train = univariate_df.iloc[:train_size, :]

x_train, y_train = pd.DataFrame(univariate_df.iloc[:train_size, 0]), pd.DataFrame(univariate_df.iloc[:train_size, 1])
x_valid, y_valid = pd.DataFrame(univariate_df.iloc[train_size:, 0]), pd.DataFrame(univariate_df.iloc[train_size:, 1])

#print(len(train), len(x_valid))


Testando LSTM


In [ ]:
from sklearn.preprocessing import MinMaxScaler
data = univariate_df.filter(['y'])

#Converter dataframe para array (numpy)
dataset = data.values

#Mudando a escala 
#scaler = MinMaxScaler(feature_range=(-1, 0))
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(dataset)
scaled_data[:10]

In [ ]:
# Defines the rolling window
look_back = 3
# Split into train and test sets
train, test = scaled_data[:train_size-look_back,:], scaled_data[train_size-look_back:,:]

def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(look_back, len(dataset)):
        a = dataset[i-look_back:i, 0]
        X.append(a)
        Y.append(dataset[i, 0])
    return np.array(X), np.array(Y)

x_train, y_train = create_dataset(train, look_back)
x_test, y_test = create_dataset(test, look_back)

# reshape input to be [samples, time steps, features]
x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))

print(len(x_train), len(x_test))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import EarlyStopping

#Build the LSTM model
model = Sequential()
early_stop = EarlyStopping(monitor='val_loss', patience = 30)

model.add(LSTM(128, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dropout(0.1))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_percentage_error') 

#Train the model
history = model.fit(x_train, y_train, batch_size=1, epochs=500, validation_data=(x_test, y_test), callbacks=[early_stop])

model.summary()

In [ ]:
# Lets predict with the model
train_predict = model.predict(x_train)
test_predict = model.predict(x_test)

# invert predictions
train_predict = scaler.inverse_transform(train_predict)
y_train = scaler.inverse_transform([y_train])

test_predict = scaler.inverse_transform(test_predict)
y_test = scaler.inverse_transform([y_test])

In [ ]:
!pip install scikit-learn==0.24

In [ ]:
# Get the root mean squared error (RMSE) and MAE
from sklearn.metrics import mean_absolute_percentage_error
score_rmse = np.sqrt(mean_squared_error(y_test[0], test_predict[:,0]))
score_mae = mean_absolute_error(y_test[0], test_predict[:,0])
score_mae = mean_absolute_percentage_error(y_test[0], test_predict[:,0])
#print(Fore.GREEN + 'RMSE: {}'.format(score_rmse))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
x_train_ticks = univariate_df.head(train_size)['ds']
y_train = univariate_df.head(train_size)['y']
x_test_ticks = univariate_df.tail(test_size)['ds']

# Plot the forecast
f, ax = plt.subplots(1)
f.set_figheight(6)
f.set_figwidth(15)

sns.lineplot(x=x_train_ticks, y=y_train, ax=ax, label='Train Set') #navajowhite
sns.lineplot(x=x_test_ticks, y=test_predict[:,0], ax=ax, color='green', label='Prediction') #navajowhite
sns.lineplot(x=x_test_ticks, y=y_test[0], ax=ax, color='orange', label='Ground truth') #navajowhite

ax.set_title(f'Previsão de vendas \n RMSE: {score_rmse:.2f}, MAPE: {score_mae:.2f}', fontsize=14)
ax.set_xlabel(xlabel='Data', fontsize=14)
ax.set_ylabel(ylabel='Vendas', fontsize=14)

plt.show()